# Prepare data for visualisation

## Libraries


In [504]:
import json
import pandas as pd
from collections import Counter
from collections import OrderedDict
import statistics
import math
from ojd_daps_skills import get_yaml_config, Path, PROJECT_DIR

## Parameters

In [671]:
# Number of occupations to include in the data visualisation
NO_OCCUPATIONS = 50

In [677]:
# End dates where we are missing job titles (these job adverts needs to be dropped)
DATES_MISSING_DATA = ['2022-06-10', '2022-07-22', '2022-08-19']

In [672]:
# To hold the final data 
final_data = []

## Paths

In [673]:
# Path to sample of job adverts
PATH_RAW_DATA = str(PROJECT_DIR)+"/inputs/dedupe_analysis_skills_sample.json"

# Path to csv which contains salaries and job titles of the adverts
PATH_SALARIES_TITLES = str(PROJECT_DIR)+"/inputs/dedupe_analysis_metadata_salaries_titles.csv"

# Path to ESCO taxonomy
PATH_ESCO = str(PROJECT_DIR)+"/inputs/esco_data_formatted.csv"

# Path to ESCO hierarchy
PATH_ESCO_HIERARCHY = str(PROJECT_DIR)+"/inputs/esco_hier_mapper.json"

# Path to prepared data
PATH_FINAL_DATA = str(PROJECT_DIR)+"/ojd_daps_skills/analysis/OJO/data_visualisation/visualisation/data/data.json"


## Open files

In [674]:
# Load sample of job adverts
with open(PATH_RAW_DATA) as f:
    raw_data = json.load(f)

In [675]:
# Load dataframe containing salaries and titles of jobs
df_salaries = pd.read_csv(PATH_SALARIES_TITLES, index_col='job_id') 

In [684]:
# Load ESCO taxonomy
df_esco = pd.read_csv(PATH_ESCO, index_col='id') 

In [685]:
# Load ESCO hierarchy
with open(PATH_ESCO_HIERARCHY) as f:
    dict_esco_all = json.load(f)

## Clean files

In [680]:
# Delete all job adverts that are missing job titles (due to a glitch)
raw_data = [one_job_advert for one_job_advert in raw_data if one_job_advert['end_date_chunk'] not in DATES_MISSING_DATA]

In [681]:
# Add salary data into raw_data
for one_advert in raw_data:
    
    min_ann_salary = df_salaries.loc[int(one_advert['job_id'])]['min_annualised_salary']
    max_ann_salary = df_salaries.loc[int(one_advert['job_id'])]['max_annualised_salary']
   
    if min_ann_salary is not None and max_ann_salary is not None:
        one_advert['salary'] = (min_ann_salary + max_ann_salary)/2
    else:
        one_advert['salary'] = None
    

In [682]:
# Add job title data into raw_data (not really used)
for one_advert in raw_data:    
    one_advert['job_title'] = df_salaries.loc[int(one_advert['job_id'])]['job_title_raw']    
    

In [686]:
# Only keep the rows of the ESCO taxonomy that contain the preferred skill label
# (to prevent duplicates)
df_esco = df_esco[df_esco['type']=='preferredLabel']

In [687]:
# Extract the level 1 skill groups for each skill
# (level 1 = the second to top skill level
# i.e. every 4th entry in the 'hierarchy_levels' column, starting at position 1)
df_esco['level_1'] = df_esco['hierarchy_levels'].apply(lambda x: x.replace("[","")
                                                                  .replace("]","")
                                                                  .replace("'","")
                                                                  .split(", ")[1::4])

## Occupations

In [696]:
# How many adverts are missing an 'occupation'?
# It's around 23% - that's why we use 'sector' instead
# And 'sector' is more accurately described as occupation groups
nan_occup_percent = 100*(sum([1 for one_advert in raw_data if isinstance(one_advert['occupation'], str)==False 
                   and math.isnan(one_advert['occupation'])])/len(raw_data))
print(str(round(nan_occup_percent))+"%")

23%


In [697]:
# Find the 50 largest occupational groups (aka 'sectors'), excluding those that are 'other' (e.g. 'other administrators')
occupations = Counter([value['sector'] for value in raw_data 
                       if 'Other' not in value['sector']]).most_common(NO_OCCUPATIONS)

In [698]:
# Add the occupations to the final data
final_data = [{'occupation': one_occupation[0], 
               'no_jobs': one_occupation[1]} 
                   for one_occupation in occupations]

## Salaries

In [701]:
# Loop over the adverts
for one_occupation in final_data:
    
    # Calculate the median salary for those adverts in that occupation
    all_salaries = [one_advert['salary'] for one_advert in raw_data if math.isnan(one_advert['salary'])==False 
                                       and one_advert['sector']==one_occupation['occupation']]
    
    # Calculate median
    median_salary = statistics.median(all_salaries)
    
    # Add the median to the final data set
    one_occupation['median_salary'] = median_salary


In [702]:
# Reorder the entries in final_data by median salary (high to low)
final_data = sorted(final_data, key=lambda d: d['median_salary'], reverse = True) 

## Cumulative jobs

In [703]:
# Add in the cumulative number of jobs across occupations 
# (this is to assist with placement in the data viz)
cumul_jobs = 0
for one_occupation in final_data:
    one_occupation['cumul_jobs'] = cumul_jobs
    cumul_jobs += one_occupation['no_jobs']

## Locations

In [704]:
# For EACH occupation add in the fraction of adverts that are for positions in London
for one_occupation in final_data:
    
    # Total number of adverts for positions in London (in that occupation)
    count_london = sum([1 for value in raw_data if value['itl_1_name']=="London" 
                        and value['sector']==one_occupation['occupation']])
    
    # Add in percetnage
    one_occupation['frac_london'] = count_london/one_occupation['no_jobs']
    

In [705]:
# For ALL occupations, calculate the fraction of adverts that are for positions in London
# (this figure is inserted manually into the data viz)

overall_frac_london = 100*sum([1 for value in raw_data if value['itl_1_name']=="London"])/len(raw_data)

print("Percentage of ALL adverts in London {}".format(overall_frac_london))


Percentage of ALL adverts in London 21.32386277769382


## Skill groups

In [710]:
# Find the maxium length of the skill groups IDs 
# so as to distinguish between these groups and 
# the individual skills which have longer IDs)
MAX_LENGTH_GROUP = max(len(key) for key in dict_esco_all)

In [711]:
# Loop over each occupation
for one_occupation in final_data:
    
    # Extract all adverts for that occupation
    raw_data_one_occup = [one_advert for one_advert in raw_data if one_advert['sector']==one_occupation['occupation']]
    
    # Calculate the percentage of adverts that contain at least one skill in the broad skill group (for each group)
    # 1. Form a dictionary of level one skill groups to store results
    # 2. Extract the ESCO IDs from the skills in each advert (both skill IDs and skill group IDs)
    # 3. Form a the list of unique Level 1 skill groups associated with one job advert
    dict_esco_l1 = extract_l1_groups(raw_data_one_occup, dict_esco_all, MAX_LENGTH_GROUP)
                
    # Add in the number of jobs to each skill entry (needed for the visualisation)
    for _, one_skill_group in dict_esco_l1.items():
        one_skill_group['no_jobs'] = one_occupation['no_jobs']

    # Save the results
    one_occupation['skills'] = [value for key, value in dict_esco_l1.items()]
        

## Save

In [712]:
with open(PATH_FINAL_DATA, 'w') as f:
    json.dump(final_data, f)

## Extra

In [726]:
# Find the most common job titles for a given sector 
# (used to rename some sectors that have vague titles)
Counter([one_advert['job_title'] for one_advert in raw_data if one_advert['sector']=='Qualified Social Worker']).most_common(10)

[('Assistant Locality Manager', 8),
 ('Supervising Social Worker', 7),
 ('Social Worker', 6),
 ('Qualified Social Worker', 6),
 ('Occupational Therapist', 5),
 ('Family Court Adviser Social Worker', 4),
 ('Qualified Social Worker - Child Protection Team', 3),
 ('Care Practitioner', 3),
 ('Qualified Social Worker - Adults Learning Disabilities', 3),
 ('Family Court Advisor', 3)]

## Functions

In [585]:
# Extract the level one ESCO skill groups for a group of adverts
def extract_l1_groups(raw_data_one_occup, dict_esco_all, MAX_LENGTH_GROUP):
    
    # Create an empty dictionary of the level 1 ESCO hierarchy to capture counts
    dict_esco_l1 = create_dict_esco_l1(dict_esco_all)    

    # Loop over each advert
    for one_advert in raw_data_one_occup:
        
        # If the advert contains at least one skill
        if one_advert['SKILL'] is not None:

            # Form a list of ESCO IDs containing 'UNIQUE skill IDs' and 'ALL mentions of skill groups'
            esco_ids = extract_esco_ids(one_advert['SKILL'], MAX_LENGTH_GROUP)

            # Extract unique level one skill groups for each ESCO ID
            level_one_groups = find_level_one_groups(esco_ids, MAX_LENGTH_GROUP, dict_esco_l1)

            # Add the level one skill groups to the dictionary of the level 1 ESCO hierarchy
            for one_group in level_one_groups:
                dict_esco_l1[one_group]['weight'] += 1
                
    # Scale the weights by the number of adverts
    for key, value in dict_esco_l1.items():
        value['percentage'] = 100*value['weight']/len(raw_data_one_occup)
    
    return dict_esco_l1


In [407]:
# Takes the whole ESCO hierarchy and creates a dictionary containing
# just the level one groups (the second to top layer in the taxonomy)
def create_dict_esco_l1(dict_esco_all):
    
    # Dictionary of the broadest skill groups
    dict_broad = {"K": "knowledge", 
                  "A": "attitudes and values",  
                  "T": "transversal competences", 
                  "S": "skills", 
                  "L": "language skills and knowledge"}

    # Dictionary of the level 1 skill groups (to store results)
    dict_esco_l1 = {key: {'name': value, 'weight': 0, 'broad': dict_broad[key[0]]} for key, value in dict_esco_all.items() if 
                (key[0]!='K' and len(key)==2) or 
                (key[0]=='K' and len(key)==3)}
    
    # Reorder
    dict_esco_l1 = OrderedDict(sorted(dict_esco_l1.items(), key=lambda x: x[1]['broad']))  
    
    return dict_esco_l1  

In [399]:
# Extract the ESCO IDs from the skills in each advert
# The IDs are a mixture of skill IDs and the IDs of groups
def extract_esco_ids(one_advert_skills, MAX_LENGTH_GROUP):
    
    ## Loop over skills in advert
    esco_ids = []
    for one_entry in one_advert_skills:

        ## One ESCO ID
        one_esco_id = one_entry[1][1]

        ## Consider storing the ID
        # If the ID refers to a skill group, store it
        if len(one_esco_id)<=MAX_LENGTH_GROUP:
            esco_ids.append(one_esco_id)
            
        # If the ID refers to an individual skill...
        else:
            # ..only store if it hasn't been captured before
            if one_esco_id not in esco_ids:
                esco_ids.append(one_esco_id)
                
    return esco_ids


In [708]:
# Find the list of unique Level 1 groups associated with one job advert
def find_level_one_groups(esco_ids, MAX_LENGTH_GROUP, dict_esco_l1):

    # Loop over each ESCO ID
    level_one_groups = []
    for one_esco_id in esco_ids:

        # If the ESCO ID refers to a SKILL GROUP
        if len(one_esco_id)<=MAX_LENGTH_GROUP:
            
            # If the group is a type of knowledge
            if one_esco_id[0]=='K':
                one_l1_group = one_esco_id[0:3] 
            else: 
                one_l1_group = one_esco_id[0:2]
                                        
            # Add level one group 
            level_one_groups.append(one_l1_group)
                
                
        # If the ESCO ID refers to an INDIVIDUAL (and unique) SKILL
        else:
            # The multiple skill groups associated with that one skill
            multiple_l1_groups = df_esco.loc[one_esco_id]['level_1']
            level_one_groups.extend(multiple_l1_groups)
                
    # Get rid of duplicate level one groups
    level_one_groups = list(set(level_one_groups))
    
    return level_one_groups
                